## Compare similarity and shrink the lemma base

In [2]:
### load lemma
import pickle
import os
data_dir = "../data/"
category = ['Alt', 'Center', 'Left', 'Right']

In [2]:
import numpy as np
from numpy import linalg as LA
def similarity_distance(v1, v2):
    similarity = np.sum(v1*v2)/(LA.norm(v1)*LA.norm(v2))
    diff = v1-v2
    distance = np.sqrt(np.sum(diff**2))
    return similarity, distance

In [34]:
from sklearn.cluster import KMeans
from nltk.cluster import KMeansClusterer
import nltk
import time
class LemmaClusteringModel():
    def __init__(self, spa_nlp, lemma):
        self.nlp = spa_nlp
        self.lemma = list(lemma)
        self.tokens = []
        self.lemma_vectors = []
        
    def vectorize(self, n_divide):
        self.tokens = []
        per_len = len(self.lemma)//n_divide
        divide = []
        last = 0
        while last < len(self.lemma):
            divide.append(self.lemma[last:last+per_len])
            last += per_len
        for d in divide:
            for t in d:
                mo = nlp(t)
                if len(mo) == 1:         
                    self.tokens.append(mo[0])
        self.lemma_vectors=[token.vector for token in self.tokens]

    def nltkkmeans(self, num_cluster, distance, iters):
        start = time.time()
        kclusterer = KMeansClusterer(num_cluster, distance=distance, repeats=iters)
        lemma_vectors = [token.vector for token in self.tokens]
        assigned_clusters = kclusterer.cluster(lemma_vectors, assign_clusters=True)
        print(time.time() - start)
        return {self.tokens[i].text: assigned_clusters[i] for i in range(len(self.tokens))}
    
    def sklearnkmeans(self, num_cluster, iters):
        ## sklearn is much faster than nltk kmeans in large data set (>10000)
        start = time.time()
        kmeans = KMeans(n_clusters=num_cluster, max_iter = iters)
        kmeans.fit(self.lemma_vectors)
        print(time.time() - start)
        return {self.tokens[i].text: kmeans.labels_[i] for i in range(len(self.tokens))}
        

In [29]:
import spacy
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner', 'tagger'])

In [19]:
import string
lemma = set()
for cat in category:
    with open(cat + "lemma.pk", "rb") as f:
        lemma_ = pickle.load(f)
        print(len(lemma_))
    les = []
    for x in list(lemma_):
        if len(x) < 10 and all([i in string.ascii_lowercase for i in x]):
            les.append(x)
    lemma.update(les)
len(lemma)

69261
112547
108956
99422


155457

In [35]:
clmodel = LemmaClusteringModel(nlp, lemma)

In [36]:
clmodel.vectorize(10)

In [40]:
len(clmodel.tokens),len(clmodel.lemma_vectors)

(155407, 155407)

In [73]:
MAX_ITER = 1000
CLUSTER = 110
assignment = clmodel.sklearnkmeans(CLUSTER, MAX_ITER)

723.2858109474182


In [74]:
g = []
for i in assignment:
    if i  == "big":
        print(assignment[i])
    if i == "huge":
        print(assignment[i])
    if i == "home":
        print(assignment[i])
    if i == "house":
        print(assignment[i])
    if i == "mother":
        print(assignment[i])
    if i == "brother":
        print(assignment[i])
    if i == "father":
        print(assignment[i])
    if i == "fat":
        print(assignment[i])
    if i == "thin":
        print(assignment[i])

98
51
69
51
105
98
98
51
101


In [65]:
for i in range(len(clmodel.lemma)):
    if clmodel.tokens[i].text == "big":
        t1 = clmodel.lemma_vectors[i]
    if clmodel.tokens[i].text == "house":
        t2 = clmodel.lemma_vectors[i]
    if clmodel.tokens[i].text == "home":
        t3 = clmodel.lemma_vectors[i]

In [66]:
print(similarity_distance(t1, t2), similarity_distance(t2, t3), similarity_distance(t1,t3))

(0.40848243, 6.450766) (0.73886245, 4.344995) (0.39892742, 6.4241996)


In [60]:
with open("clusterAssignmen100.pk", "wb") as f:
    pickle.dump(assignment, f)

## Assignment for each datum

In [3]:
with open("clusterAssignmen100.pk", "rb") as f:
    assignment = pickle.load(f)